# MSA Phase 2 Part 3 - Tune Hyperparameters

## 1. Connect to workspace

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.52.0 to work with MSA-Phase2-Azure


## 2. Prepare a training script

In [2]:
import os, shutil

# Create a folder for the training script
experiment_folder = 'training-hyperdrive'
os.makedirs(experiment_folder, exist_ok=True)

shutil.copy('../1. Analysis and Preprocessing/preprocessed_datasets/market_segmentation.csv', os.path.join(experiment_folder, "market_segmentation.csv"))

print('Folder ready.')

Folder ready.


In [21]:
%%writefile $experiment_folder/training.py

# Import libraries
import argparse, joblib, os
from azureml.core import Run
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer

# Get the experiment run context
run = Run.get_context()

# Get script arguments
parser = argparse.ArgumentParser()

# Set regularization and solver hyperparameters
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
parser.add_argument('--solver', type=str, dest='solver', default='lbfgs')
args = parser.parse_args()
reg = args.reg
solver = args.solver

# Log Hyperparameter values
run.log('reg',  np.float(args.reg))
run.log('solver', args.solver)

# load the market segmentation dataset
print("Loading Data...")
market_segmentation = pd.read_csv('market_segmentation.csv')

# Separate features and labels
X, y = market_segmentation.drop(columns="Segmentation"), market_segmentation.Segmentation

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# Train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver=solver).fit(X_train, y_train)

# Calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# Calculate AUC
label_binarizer = LabelBinarizer().fit(y_train)
y_onehot_test = label_binarizer.transform(y_test)
y_scores = model.predict_proba(X_test)
for class_of_interest in ["A", "B", "C", "D"]:
    class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
    auc = roc_auc_score(y_onehot_test[:,class_id],y_scores[:,class_id])
    print(f'AUC {class_of_interest} vs rest: ' + str(auc))
    run.log(f'AUC {class_of_interest} vs rest', np.float(auc))

# Save the model in the run outputs
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')

run.complete()

Overwriting training-hyperdrive/training.py


## 3. Create compute

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "aguo921"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)
    

Found existing cluster, use it.


## 4. Run a hyperparameter tuning experiment

In [6]:
%%writefile $experiment_folder/hyperdrive_env.yml
name: batch_environment
dependencies:
- python=3.6.2
- scikit-learn
- pandas
- numpy
- pip
- pip:
  - azureml-defaults

Overwriting training-hyperdrive/hyperdrive_env.yml


In [17]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.train.hyperdrive import GridParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice

# Create a Python environment for the experiment
hyper_env = Environment.from_conda_specification("experiment_env", experiment_folder + "/hyperdrive_env.yml")

# Create a script config
script_config = ScriptRunConfig(
    source_directory=experiment_folder,
    script='training.py',
    environment=hyper_env,
    compute_target = training_cluster
)

# Sample a range of parameter values
params = GridParameterSampling(
    {
        '--reg': choice(0.3, 0.4, 0.5, 0.6),
        '--solver': choice('newton-cg', 'saga')
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(
    run_config=script_config, 
    hyperparameter_sampling=params, 
    policy=None, # No early stopping policy
    primary_metric_name='Accuracy', # Find the highest Accuracy metric
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs=12, # Restict the experiment to 16 iterations
    max_concurrent_runs=2 # Run up to 2 iterations in parallel
) 

# Run the experiment
experiment = Experiment(workspace=ws, name='segmentation-hyperdrive')
run = experiment.submit(config=hyperdrive)

# Show the status in the notebook as the experiment runs
run.wait_for_completion()

{'runId': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca',
 'target': 'aguo921',
 'status': 'Completed',
 'startTimeUtc': '2023-08-01T09:43:30.959306Z',
 'endTimeUtc': '2023-08-01T09:48:06.803121Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1f91b411-324e-4cf5-83ca-86bd82e4bf98',
  'user_agent': 'python/3.11.3 (Windows-10-10.0.22621-SP0) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.52.0',
  'space_size': '8',
  'score': '0.4968179889690284',
  'best_child_run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_4',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-train

## 5. Determine the best performing run

In [18]:
# Print all child runs, sorted by the primary metric
for child_run in run.get_children_sorted_by_primary_metric():
    print(child_run)

# Get the best run, and its metrics and arguments
best_run = run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
script_arguments = best_run.get_details() ['runDefinition']['arguments']
print('Best Run Id: ', best_run.id)
print(' -AUC A vs rest:', best_run_metrics['AUC A vs rest'])
print(' -AUC B vs rest:', best_run_metrics['AUC B vs rest'])
print(' -AUC C vs rest:', best_run_metrics['AUC C vs rest'])
print(' -AUC D vs rest:', best_run_metrics['AUC D vs rest'])
print(' -Accuracy:', best_run_metrics['Accuracy'])
print(' -Arguments:',script_arguments)

{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_5', 'hyperparameters': '{"--reg": 0.5, "--solver": "saga"}', 'best_primary_metric': 0.4968179889690284, 'status': 'Completed'}
{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_4', 'hyperparameters': '{"--reg": 0.5, "--solver": "newton-cg"}', 'best_primary_metric': 0.4968179889690284, 'status': 'Completed'}
{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_7', 'hyperparameters': '{"--reg": 0.6, "--solver": "saga"}', 'best_primary_metric': 0.4963937208315656, 'status': 'Completed'}
{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_6', 'hyperparameters': '{"--reg": 0.6, "--solver": "newton-cg"}', 'best_primary_metric': 0.4963937208315656, 'status': 'Completed'}
{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_2', 'hyperparameters': '{"--reg": 0.4, "--solver": "newton-cg"}', 'best_primary_metric': 0.4959694526941027, 'status': 'Completed'}
{'run_id': 'HD_2f8eb167-ea06-483f-82cf-4dc30b37c5ca_3', 'hyperparameters': '{"--reg": 0.4,

In [19]:
from azureml.core import Model

# Register model
best_run.register_model(
    model_path='outputs/model.pkl', model_name='segmentation-model',
    tags={'Training context':'Hyperdrive'},
    properties={
        'AUC A vs rest': best_run_metrics['AUC A vs rest'],
        'AUC B vs rest': best_run_metrics['AUC B vs rest'],
        'AUC C vs rest': best_run_metrics['AUC C vs rest'],
        'AUC D vs rest': best_run_metrics['AUC D vs rest'],
        'Accuracy': best_run_metrics['Accuracy']
    }
)

# List registered models
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

segmentation-model version: 5
	 Training context : Hyperdrive
	 AUC A vs rest : 0.7173467464996174
	 AUC B vs rest : 0.6771250706291696
	 AUC C vs rest : 0.7548471920284576
	 AUC D vs rest : 0.8663999845277084
	 Accuracy : 0.4968179889690284


segmentation-model version: 4
	 Training context : Hyperdrive
	 AUC A vs rest : 0.7173467464996174
	 AUC B vs rest : 0.6771250706291696
	 AUC C vs rest : 0.7548471920284576
	 AUC D vs rest : 0.8663999845277084
	 Accuracy : 0.4968179889690284


segmentation-model version: 3
	 Training context : Hyperdrive
	 AUC A vs rest : 0.7111122006745038
	 AUC B vs rest : 0.6643216130215407
	 AUC C vs rest : 0.7541089423407565
	 AUC D vs rest : 0.8669461195860793
	 Accuracy : 0.49087823504454814


segmentation-logistic-regression version: 2


segmentation-model version: 2
	 Training context : Script
	 AUC A vs rest : 0.7113390862280469
	 AUC B vs rest : 0.6641015473983684
	 AUC C vs rest : 0.754124484439445
	 AUC D vs rest : 0.8668678371578138
	 Accuracy : 0.4

## 6. Screenshots of final run

Below are screenshots of the Azure interface from my final hyperparameter tuning run.

![](images/accuracy-vs-time.png)

The accuracy of the best child over time as we sample different sets of hyperparameters improves over time as expected.

![](images/parallel-coordinates.png)

Below is a parallel coordinates chart showing the effect of the regularisation rate and solver on the accuracy of the model. We find that the newton-cg and saga solver give the same accuracy and that a regularisation rate of 0.5 gives the best accuracy.

![](images/best-model-metrics.png)

The best hyperparameter tuned model has an accuracy of almost 50%, which means it correctly segments almost half of the customers. It has the highest AUC score when classifying customers in group D and the lowest AUC score when classifying customers in group B.